# WR template in pyHarm

This notebook is bult in order to provide a example into how to implement WR as a connector in pyHarm.

- Single connector that grabs 3 dofs (slave, master, force) : 
    - modify residual in order to obtain the WR effect
    - Just set the force dofs as being nonlinear
- In the input, add a `force_dof` as a `onlydofs` substructure that will hold the necessary dofs

**/!\ : NEED PYHARM >= 1.1.2 to work [current release is 1.1.3]**

In [9]:
import pyHarm
import numpy as np
from pyHarm.BaseUtilFuncs import pyHarm_plugin # to dynamically plug any changes to the created class in the notebook
TEST = True

# buld simple one dof linear system : 

M = np.ones((1,1))
K = 1e2*np.ones((1,1))
C = 1e-1 * np.ones((1,1))

sub_d = dict(
    onedof = dict(
        type = 'substructure',
        matrix = dict(
            M=M,
            K=K,
            C=C
        ),
        ndofs=1 # number of dof per node
    ),
    wrforces = dict( # this is only needed for the WR case 
        # (note : ok for testing/bench/dev, in prod need dynamic creation of dof when WR elemetns are instanciated)
        type = 'onlydofs',
        nnodes = 1,
        ndofs=1, # number of dof per node
        nmodes=0 # number of dof per node
    ),
)

sys_d = dict(
    type='Base',\
    nh = [1],
    nti = 2**6
)

from pyHarm.Systems.FactorySystem import generateSystem

test_sys = generateSystem(
    name_system='Base',
    datas=dict(substructures=sub_d, system=sys_d)
)
# THE LOG BEHAVING WEIIIRD is normal ... well not really is a bug due to kernel not being restarded in juyter notebooks
test_sys.edf # this shows what dofs are in your system !

[   INFO   ] -- 17:15:29 -- # SYSTEM INITIALIZED :
                         |  Total dof number  |    4     |
                         |Substructures|Parts |    2     |
                         |Connectors|Elements |    1     |
                         |     Kinematics     |    0     |
                         |     Harmonics      |   [1]    |
                         |     Time steps     |    64    |

[   INFO   ] -- 17:15:29 -- # SYSTEM INITIALIZED :
                         |  Total dof number  |    4     |
                         |Substructures|Parts |    2     |
                         |Connectors|Elements |    1     |
                         |     Kinematics     |    0     |
                         |     Harmonics      |   [1]    |
                         |     Time steps     |    64    |



,sub,harm,cs,node_num,dof_num,PoM,NL,KC
0,onedof,1,c,0,0,p,0,0
1,wrforces,1,c,0,0,p,0,0
2,onedof,1,s,0,0,p,0,0
3,wrforces,1,s,0,0,p,0,0


In [10]:
# run the linerar system by adding some forcing

# we need to add something on the last dof for hte linear system otherwise ... not happy with the matrices : 
# here a kinematic condition forcing wrforces=1 --> THIS IS JUST FOR LINEAR

In [11]:
# buid the new connector that does ... whatever but takes 3dofs using the grabbers
from pyHarm.DofGrabber import dof_grabber
if TEST :  
    index_forces = dof_grabber(
        edf=test_sys.edf,
        sub="wrforces",
        node=0,
        dof=0
    )
    print(test_sys.edf.loc[index_forces])
#### comment : there is a bunch of grabbing functions for getting all the dofs from a node, etc ... all in DofGrabber module


        sub  harm cs  node_num  dof_num PoM  NL  KC
1  wrforces     1  c         0        0   p   0   0
3  wrforces     1  s         0        0   p   0   0
